# Photogrammetry Example

In [ ]:
import logging
logging.basicConfig()
logging.getLogger("cedalion").setLevel(logging.DEBUG)

import cedalion.io
import cedalion.plots
from cedalion.geometry.photogrammetry.processors import ColoredStickerProcessor
from cedalion.datasets import get_photogrammetry_example_scan
import xarray as xr
import pyvista as pv

xr.set_options(display_expand_data=False);

## Choose between interactive and static 3D plots

In [ ]:
pv.set_jupyter_backend("static")  # uncomment for static rendering
#pv.set_jupyter_backend("client")  # uncomment for interactive rendering
#pv.set_jupyter_backend("html")

Use `cedalion.io.read_einstar_obj` to read the textured triangle mesh produced by the Einstar scanner.

In [ ]:
example_scan_fname = get_photogrammetry_example_scan()
s = cedalion.io.read_einstar_obj(example_scan_fname)

Processors are meant to analyze the textured mesh and extract positions. The ColoredStickerProcessor searches for colored circular areas. The colors must be specified by their ranges in hue and value. These can for example be found by usig a color pipette tool on the texture file.

In the following to classes of stickers are searched: "O(ptodes)" in blue and "L(andmarks" in yellow.

In [ ]:
processor = ColoredStickerProcessor(
    colors={
        "O" : ((0.11, 0.21, 0.8, 1)), # (hue_min, hue_max, value_min, value_max)
        "L" : ((0.25, 0.37, 0.35, 0.6))
    }
)

In [ ]:
sticker_centers, normals, details = processor.process(s, details=True)

In [ ]:
display(sticker_centers)

Visualize the surface and extraced results.

In [ ]:

plt = pv.Plotter()
cedalion.plots.plot_surface(plt, s, opacity=1.0)
cedalion.plots.plot_labeled_points(plt, sticker_centers, color="r")
cedalion.plots.plot_vector_field(plt, sticker_centers, normals)
plt.show()

The details object is meant as a container for debuging information. It also provides plotting functionality.The following scatter plot shows the vertex colors in the hue-value plane in which the vertex classification operates.

In [ ]:
details.plot_vertex_colors()

The following plots show for each cluster (tentative group of sticker vertices) The vertex positions perpendicular to the sticker normal as well as the minimum enclosing circle which is used to find the sticker's center.

In [ ]:
details.plot_cluster_circles()

Finally, to get from the sticker centers to the scalp coordinates we have to subtract the lenght of the optodes in the direction of the normals:

In [ ]:
optode_length = 22.6 * cedalion.units.mm

scalp_coords = sticker_centers.copy()
mask_optodes = sticker_centers.group == 'O'
scalp_coords[mask_optodes] = sticker_centers[mask_optodes] - optode_length*normals[mask_optodes]

In [ ]:
display(scalp_coords)

In [ ]:
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, s, opacity=0.3)
cedalion.plots.plot_labeled_points(plt, sticker_centers, color="r")
cedalion.plots.plot_labeled_points(plt, scalp_coords, color="g")
cedalion.plots.plot_vector_field(plt, sticker_centers, normals)
plt.show()

**TBD: The found landmark and optode positions must still be matched to a montage in order to distinguish between sources and detectors and to assign the correct labels.**